In [0]:
import pandas as pd

weather = pd.read_csv("../Data/madrid_weather.csv", parse_dates=["time"])

weather.head()

### EDA & Smoothing

In [0]:
weather.set_index("time").loc[:, "temperature"].plot()

In [0]:
# 24 = daily moving average
# (24 * 30) = 30-day moving average
# (24 * 365) = yearly moving average


weather.set_index("time").loc[:, "temperature"].rolling((24 * 30)).mean().plot();

### Decomposition

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose

taco_stands = pd.read_csv("../Data/taco_stands.csv", parse_dates=["Date"])

taco_stands.head()

In [0]:
taco_stands.set_index("Date").plot()

In [0]:
seasonal_decompose(taco_stands.set_index("Date")).plot();

In [0]:
from statsmodels.graphics.tsaplots import plot_acf

# Need to difference our series to remove trend, slicing off missing value to plot

plot_acf(taco_stands["Taco Stands in LA"].diff()[1:]);

### Linear Regression

In [0]:
monthly_weather =(
    weather
    .set_index("time")
    .resample("M")
    .agg({"temperature": "mean"})
    .reset_index()
)

monthly_weather.tail()

In [0]:
# Create Trend and Seasonal Variables

monthly_weather = monthly_weather.assign(
    trend = monthly_weather.index,
    month = monthly_weather["time"].dt.month.astype("object")
)

monthly_weather.head()

In [0]:
# Get Seasonal Dummies

monthly_weather = pd.get_dummies(monthly_weather, drop_first=True)

monthly_weather.tail()

In [0]:
# Split Data Based on Time

weather_train = monthly_weather.set_index("time").loc[:"2021-09"]
weather_test = monthly_weather.set_index("time").loc["2021-10-31":]

weather_test

In [0]:
# Fit Regression

import statsmodels.api as sm

y = weather_train["temperature"]
X = sm.add_constant(weather_train.iloc[:, 1:]).drop("trend", axis=1)

model = sm.OLS(y, X).fit()

model.summary()

In [0]:
# Make Predictions

y_test = weather_test["temperature"]
X_test = sm.add_constant(weather_test.iloc[:, 1:]).drop("trend", axis=1)

prediction = model.predict(X_test)

prediction

In [0]:
# Predictions vs. Actual DataFrame

pred_df = pd.DataFrame({"actual": y_test, "predicted": prediction})

pred_df

In [0]:
# Performance Metrics

from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae

print(mape(pred_df["actual"], pred_df["predicted"]))
print(mae(pred_df["actual"], pred_df["predicted"]))

In [0]:
## Forecast Chart

pred_df.plot(ylim=0)

## Facebook Prophet

In [0]:
# Reshape Dataframe into Facebook Prophet Data

monthly_weather = (
    weather
    .set_index("time")
    .resample("M")
    .agg({"temperature": "mean"})
    .reset_index()
    .rename({"time": "ds", "temperature": "y"}, axis=1)
)

monthly_weather.head()

In [0]:
# Split Data

monthly_weather_train = monthly_weather[:-4]
monthly_weather_test = monthly_weather[-4:]

monthly_weather_test

In [0]:
#!conda install prophet --y

from prophet import Prophet

m = Prophet()
m.fit(monthly_weather_train)

In [0]:
future = m.make_future_dataframe(periods=4, freq="M")

forecast = m.predict(future)

m.plot(forecast);

In [0]:
fig = m.plot_components(forecast)

In [0]:
# Plot Forecast Vs. Actual

import seaborn as sns

(monthly_weather_test
 .assign(predictions = m.predict(future)["yhat"])
 .set_index("ds")
 .plot(ylim=0)
)

sns.despine()

In [0]:
# Performance Metrics

print(f" MAPE: {mape(monthly_weather_test['y'], m.predict(future).iloc[-4:, -1])}")
print(f" MAE: {mae(monthly_weather_test['y'], m.predict(future).iloc[-4:, -1])}")